In [ ]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

In [ ]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = config('OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')

## Loading Answer Generation Model

In [ ]:
from models.llm import LLM

model = LLM('gpt-3')
llm = model.load_model()

## Input Document Ingestion

In [ ]:
input_txt = """"Analysts have valued Reliance Industries Ltd’s new energy business at as much as Rs 2.36 lakh crore (Rs 350 per share), exceeding the market value of half of the benchmark Nifty index’s constituents.

There are 25 companies in the 50-member Nifty index with a market capitalisation of less than Rs 2.36 lakh crore, according to a Moneycontrol analysis.

Investor confidence in India’s largest conglomerate has surged on increased optimism about the prospects of the new energy unit, the expected demerger of the retail business, and increased free cash flow as capital expenditure needs ebb.

The RIL stock has touched a new high, surpassing a market capitalisation milestone of Rs 20 lakh crore earlier this week after remaining within a range for the past three years. In January, mutual funds boosted their stake in the company, buying shares worth Rs 3,300 crore. As of January 31, RIL had the second-highest weight on the Nifty index after HDFC Bank.
Analysts expect investors to remain bullish on the Reliance stock

New energy
Deven Choksey, managing director of DRChoksey FinServ, said that RIL already has a 5-10-year head start, while state-run companies are just showing interest in the renewables segment.

“Reliance has moved considerably ahead in the renewables business. They have already created a complete ecosystem for producing green hydrogen. They have manufactured solar panels and have solar farms in place. They have a port-based presence in Jamnagar. They are probably upgrading new technology in the electrolyser process,” he said.

Among brokerages, Goldman Sachs and Jefferies have assigned a value of more than Rs 300 to the new energy business. Nomura has ascribed the lowest value at Rs 182.

The biggest trigger for this segment will be the commencement of the new energy giga complex in Gujarat. Following the announcement of fiscal third-quarter earnings, RIL chairman Mukesh Ambani said, “New Energy Giga Complex is all set to be commissioned in the second half of CY24.”

The complex will include five giga factories for photovoltaic panels, fuel cell systems, green hydrogen, energy storage, and power electronics.

Retail and Jio demerger
“RIL is on a strong wicket for the next 3-5 years. The traditional E&P (exploration and production) and refining business will continue to generate cash flow, but the real value to RIL stock will come from new-age businesses,” according to a fund manager, who declined to be named as he owns the stock in his portfolio.

At the current fair value, brokerages are ascribing Rs 1,300-1,400 to the retail business, almost half the entire stock price. With an overall store count of 18,774 stores, the retail business aims to reach every corner of India.

Apart from that, the company has aggressively acquired companies across categories, from beverage maker Campa Cola to clothing company Ed-a-Mamma to online pharmacy Netmeds. It is also taking on the likes of Nykaa with the launch of Tira, an omnichannel beauty platform.

A clutch of investors such as KKR, Abu Dhabi Investment Authority and Qatar Investment Authority put in fresh investments in Reliance Retail in 2023, boosting expectations of an IPO of the retail business soon.

“Investments for Jio and Retail platforms have already been completed, and their B2C models are growing at around 20 percent. These two companies coming up for separate listing will create new market value for shareholders. That is the next trigger that investors will be looking out for. The stock is full of possibilities in every aspect,” Choksey said.

For Jio, another big trigger is the bump up in ARPU (average revenue per user), which can be expected from a tariff hike post-election. Nomura expects ARPU to cross Rs 200 in FY25.

Free cash flow
Over the past few years, RIL was in aggressive capex mode for the retail segment and Jio’s 5G business. This is slowly tapering off. In Q3 FY24, the conglomerate’s capex stood at Rs 30,102 crore versus Rs 38,815 crore in the quarter-ago period.

“Reliance has operated at material negative FCF (free cash flow) for the last three years, driven by spending in telecom. As that fades, with an EBITDA run-rate of $20 billion a year, Reliance is expected to deliver positive free cash flow for the next three years,” according to brokerage JPMorgan.

Disclosure: Moneycontrol is a part of the Network18 group. Network18 is controlled by Independent Media Trust, of which Reliance Industries is the sole beneficiary.

Disclaimer: The views and investment tips expressed by experts on Moneycontrol.com are their own and not those of the website or its management. Moneycontrol.com advises users to check with certified experts before taking any investment decisions.
"""

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[" \n\n ", " \n "],
    chunk_size=400,
    chunk_overlap=0,
    length_function=len,
)

In [ ]:
from langchain.docstore.document import Document

texts = text_splitter.split_text(input_txt)
docs = [Document(page_content=t) for t in texts]

## Summary Generation

In [ ]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text",
    # verbose=True,
)

In [ ]:
result = stuff_chain.invoke(docs)
result['output_text']